In [ ]:
# here only executting stuff. no functions or classes

In [ ]:
## um die models und functions hier nutzen zu können, muss man die datein als module importieren. 
## das geht auch wenn das notebeooks sind, ist aber deutlich einfacher, wenn es einfach .py dateien sind.

In [ ]:
!pip install -r requirements.txt

## CycleGAN Maps 


In [1]:
#!/usr/bin/env python3
#-*- coding:utf-8 -*-
#$ -l cuda=1 # remove this line when no GPU is needed!
#$ -q all.q # do not fill the qlogin queue
#$ -cwd # start processes in current working directory
#$ -V # provide environment variables to processes
#$ -t 1-8 # start 8 instances: to train different models in parallel
#Cluster settings, 

try:
    model_param_id = int(os.environ['SGE_TASK_ID'])
except:
    print("no SGE_TASK_ID set, choosing default model parameters ")
    model_param_id = 0 #param_train_cycle_list[0] should be default model params

import importlib
import models
import functions 
import torch
import torch.nn as nn
importlib.reload(models)
importlib.reload(functions)
import numpy as np
#different import way for cluster
'''
import imp
models = imp.load_source('models', './models.py')
functions = imp.load_source('functions', './functions.py')
'''

# init CycleGAN
genA2B = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
genB2A = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
discA  = models.Discriminator(input_nc=3)
discB  = models.Discriminator(input_nc=3)
classifier = models.Classifier().net
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
databaseName = "cityscapes"
root_path_data = "./data/"+databaseName
root_path_checkpoints = "./checkpoints/"+databaseName

targetEpoch = 50
saveAt = 1 #how often to store the model at training
print("the device ist at")
print(device)

if device == torch.device("cuda"):
    #debug
    print("should load models to cuda")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch)))
else:
    #debug
    print("should load models to cpu")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch),map_location=torch.device('cpu')))

cycle  = models.CycleGAN(genA2B, genB2A, discA, discB, classifier, device, root_path_data, root_path_checkpoints)


param_train1  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l10", resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5)) # default
param_train2  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l5" , resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train3  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l10", resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train4  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l5" , resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train5  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l10", resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5))
param_train6  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l5" , resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train7  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l10", resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train8  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l5" , resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train_cycle_list = [param_train1, param_train2, param_train3, param_train4, param_train5, param_train6, param_train7, param_train8]

# paramter to load the background error verison. since this extension needed major changes in training/loaded training was done locally.
param_back_err = models.Param(name ="background_default", epochs = 80)

# paramters needed to call eval_testset() function
param_eval_testset = models.Param(channels = 3, size= 256) 

# parameters used for training the classifier
param_train_classifier = models.Param(channels = 3, epochs = targetEpoch, size = 256, name = "classifier_test")

no SGE_TASK_ID set, choosing default model parameters 
the device ist at
cuda
should load models to cuda
device == cuda -> False


## Train Model

In [ ]:
param_train_cycle = param_train_cycle_list[model_param_id-1]
cycle.train(param_train_cycle)   

## Generate Results

In [ ]:
#for every model --TODO
for paramItem in param_train_cycle_list:
    #load model
    cycle.load_cycle_nets(epoch = 50, model_name = paramItem.name)
    print(device)
    #generate for every test image some output image
    cycle._create_evalset_paired(paramItem,subfolder=paramItem.name)
    #calculate error with expected image

#compare errors between models

#run pix2pix ? and compare it with that?


## Calculate Error for each model

In [5]:
#MSE? SSIM? FCN? 
import os
from PIL import Image
import torchvision.transforms as transforms


losses= {}
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    subfolder = paramItem.name
    losses[paramItem.name]={}
    for source in source_domains:  
        losses[paramItem.name][source]=[]
        folder=root_path_data + "/eval{}/{}/".format(source,subfolder)
        folderSize = int(len(os.listdir(folder)) / 3)
        for i in range(0,folderSize):
            gen="{}{}_generated.jpg".format(folder, i)
            exp="{}{}_expected.jpg".format(folder, i)
            gen=Image.open(gen)
            exp=Image.open(exp)
            loss=functions.calcMSE(gen,exp)
            losses[paramItem.name][source].append(loss)
    

## display losses

In [6]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} \t source:{} \t std:{:.4f} \t sum:{:.2f} \t overall:{:.2f}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 	 source:A 	 std:0.0147 	 sum:20.06 	 overall:37.00
param:cycle_r9_advMSE_l10 	 source:B 	 std:0.0187 	 sum:16.94 	 overall:37.00
param:cycle_r9_advMSE_l5 	 source:A 	 std:0.0143 	 sum:25.65 	 overall:47.33
param:cycle_r9_advMSE_l5 	 source:B 	 std:0.0219 	 sum:21.68 	 overall:47.33
param:cycle_r9_advL1_l10 	 source:A 	 std:0.0152 	 sum:24.43 	 overall:50.35
param:cycle_r9_advL1_l10 	 source:B 	 std:0.0246 	 sum:25.93 	 overall:50.35
param:cycle_r9_advL1_l5 	 source:A 	 std:0.0140 	 sum:23.72 	 overall:48.56
param:cycle_r9_advL1_l5 	 source:B 	 std:0.0244 	 sum:24.84 	 overall:48.56
param:cycle_r5_advMSE_l10 	 source:A 	 std:0.0146 	 sum:25.27 	 overall:50.48
param:cycle_r5_advMSE_l10 	 source:B 	 std:0.0231 	 sum:25.22 	 overall:50.48
param:cycle_r5_advMSE_l5 	 source:A 	 std:0.0137 	 sum:24.20 	 overall:45.66
param:cycle_r5_advMSE_l5 	 source:B 	 std:0.0224 	 sum:21.46 	 overall:45.66
param:cycle_r5_advL1_l10 	 source:A 	 std:0.0143 	 sum:24.65 	 overall:49.

In [15]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} source:{} std:{:2f} sum:{:0f} overall:{}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 source:A std:0.031439 sum:67.284000 overall:151.08299999999997
param:cycle_r9_advMSE_l10 source:B std:0.042867 sum:83.799000 overall:151.08299999999997
param:cycle_r9_advMSE_l5 source:A std:0.030743 sum:67.930000 overall:148.739
param:cycle_r9_advMSE_l5 source:B std:0.042452 sum:80.809000 overall:148.739
param:cycle_r9_advL1_l10 source:A std:0.030743 sum:67.930000 overall:148.739
param:cycle_r9_advL1_l10 source:B std:0.042452 sum:80.809000 overall:148.739
param:cycle_r9_advL1_l5 source:A std:0.030743 sum:67.930000 overall:148.739
param:cycle_r9_advL1_l5 source:B std:0.042452 sum:80.809000 overall:148.739
param:cycle_r5_advMSE_l10 source:A std:0.030743 sum:67.930000 overall:148.739
param:cycle_r5_advMSE_l10 source:B std:0.042452 sum:80.809000 overall:148.739
param:cycle_r5_advMSE_l5 source:A std:0.030743 sum:67.930000 overall:148.739
param:cycle_r5_advMSE_l5 source:B std:0.042452 sum:80.809000 overall:148.739
param:cycle_r5_advL1_l10 source:A std:0.030743 sum:6

In [ ]:
bla
for paramItem in param_train_cycle_list:
    print(type(paramItem.name))
    print(type("hallo"))

In [36]:
#calculate differences between 50 and 100
losses50 = np.load(root_path_data+"/pairedlosses50.npy",allow_pickle=True).item()
losses100 = np.load(root_path_data+"/pairedlosses100.npy",allow_pickle=True).item()
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ50 = np.array(losses50[paramItem.name][source]).sum()
        std50 = np.std(losses50[paramItem.name][source])
        overall50 = np.array(losses50[paramItem.name]['A']).sum() + np.array(losses50[paramItem.name]['B']).sum()
        summ100 = np.array(losses100[paramItem.name][source]).sum()
        std100 = np.std(losses100[paramItem.name][source])
        overall100 = np.array(losses100[paramItem.name]['A']).sum() + np.array(losses100[paramItem.name]['B']).sum()
        diffstd = std100-std50
        diffsumm= summ100-summ50
        diffoverall= overall100-overall50
        print("param:{}\n source:{} \n std:{:2f}({:2f}) \n sum:{:0f}({:0f}) \n overall:{}({:2f})".format(paramItem.name,source,std100,diffstd,summ100,diffsumm,overall100,diffoverall))

param:cycle_r9_advMSE_l10
 source:A 
 std:0.020727(-0.000401) 
 sum:42.331000(-2.137000) 
 overall:200.31400000000002(-7.328000)
param:cycle_r9_advMSE_l10
 source:B 
 std:0.027450(-0.000226) 
 sum:157.983000(-5.191000) 
 overall:200.31400000000002(-7.328000)
param:cycle_r9_advMSE_l5
 source:A 
 std:0.019033(0.000683) 
 sum:41.606000(-0.273000) 
 overall:198.77599999999998(-13.650000)
param:cycle_r9_advMSE_l5
 source:B 
 std:0.027498(-0.000500) 
 sum:157.170000(-13.377000) 
 overall:198.77599999999998(-13.650000)
param:cycle_r9_advL1_l10
 source:A 
 std:0.019033(-0.001891) 
 sum:41.606000(-2.063000) 
 overall:198.77599999999998(-2.637000)
param:cycle_r9_advL1_l10
 source:B 
 std:0.027498(0.000904) 
 sum:157.170000(-0.574000) 
 overall:198.77599999999998(-2.637000)
param:cycle_r9_advL1_l5
 source:A 
 std:0.018212(0.000460) 
 sum:40.860000(-4.238000) 
 overall:205.512(3.300000)
param:cycle_r9_advL1_l5
 source:B 
 std:0.027137(0.000597) 
 sum:164.652000(7.538000) 
 overall:205.512(3.300000

In [ ]:
losses50.item()

In [19]:
type(losses)

dict

In [40]:
model_param_id

0